In [18]:
%pip install pytesseract
%pip install PIL
%pip install cv2

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pytesseract
from pytesseract import Output
from PIL import Image
import cv2

## OCR for word-level segmentation

In [25]:
image = cv2.imread('images/book2.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale

In [41]:
data = pytesseract.image_to_data(gray, output_type=Output.DICT)
print(data)

{'level': [1, 2, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2], 'par_num': [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], 'line_num': [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 1, 1, 1, 1, 1, 1], 'word_num': [0, 0, 0, 0, 1, 2, 3

In [44]:
n_boxes = len(data['level'])
for i in range(n_boxes):
    # filter out weakest confidence recognitions so bounding box around everything isn't drawn
    if int(data['conf'][i]) > 10: # out of 100
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 5)  # green rectangle drawn

In [45]:
output_file = 'images/book2_output.png'
cv2.imwrite(output_file, image)

True

## OCR for sentence-level segmentation

In [29]:
def get_surrounding_words(words, word_index, radius=10):
    start_index = max(word_index - radius, 0)
    end_index = min(word_index + radius + 1, len(words))
    return words[start_index:word_index], words[word_index + 1:end_index]

In [30]:
# collect all words and their bounding boxes
words_and_boxes = []

for i in range(len(data['text'])):
    if int(data['conf'][i]) > 10:
        word = data['text'][i]
        bounding_box = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        words_and_boxes.append((word, bounding_box))

# search for the desired word, e.g., banal
desired_word = 'banal'
desired_word_index = None
for index, (word, box) in enumerate(words_and_boxes):
    if word.lower() == desired_word.lower():
        desired_word_index = index
        break

In [38]:
if desired_word_index is not None:
    before_words, after_words = get_surrounding_words(words_and_boxes, desired_word_index, 10)

    # desired word
    x, y, w, h = words_and_boxes[desired_word_index][1]
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 5) # this is BGR (Blue, Green, Red) format!

    # words before the desired word
    for _, box in before_words:
        x, y, w, h = box
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 5)

    # words after the desired word
    for _, box in after_words:
        x, y, w, h = box
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 5)
else:
    print("desired word not found in the text.")

In [39]:
output_file = 'images/book2_context.png'
cv2.imwrite(output_file, image)

True